In [1]:
import os
os.environ['OMP_NUM_THREADS'] = str(os.cpu_count())

import numpy as np
import matplotlib.pyplot as plt
import gym
import random

In [2]:
from IPython.display import display, HTML, Video
from matplotlib import animation


In [3]:
def display_frames_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                       interval=20)
    return HTML(anim.to_jshtml())

In [4]:
from collections import namedtuple

Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('是名稱A', 100)

print(Tr_object)
print(Tr_object.value_b)

tr(name_a='是名稱A', value_b=100)
100


In [5]:
from collections import namedtuple

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


In [6]:
ENV = 'CartPole-v0'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 500

In [7]:
#from torch import nn
#import torch.nn.functional as F
#
#class Net(nn.Module):
#    def __init__(self, n_in, n_mid, n_out):
#        super().__init__()
#        self.fc1 = nn.Linear(n_in, n_mid)
#        self.fc2 = nn.Linear(n_mid, n_mid)
#        self.fc3 = nn.Linear(n_mid, n_out)
#        
#    def forward(self, x):
#        h1 = F.relu(self.fc1(x))
#        h2 = F.relu(self.fc2(h1))
#        output = self.fc3(h2)
#        return output
#    

In [14]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_in, n_mid, n_out):
        super().__init__()
        self.fc1 = nn.Linear(n_in, n_mid)
        self.fc2 = nn.Linear(n_mid, n_mid)
        
        self.fc3_adv = nn.Linear(n_mid, n_out)
        self.fc3_v = nn.Linear(n_mid, 1)
    
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        
        adv = self.fc3_adv(h2)
        val = self.fc3_v(h2).expand(-1, adv.size(1))
        
        output = val + adv - adv.mean(1, keepdim=True).expand(-1, adv.size(1))
        return output

In [15]:
class ReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.index = 0
        
    def push(self, state, action, state_next, reward):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
            
        self.memory[self.index] = Transition(state, action, state_next, reward)
        
        self.index = (self.index + 1) % self.capacity
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [16]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.memory = ReplayMemory(CAPACITY)
        
        n_in, n_mid, n_out = num_states, 32, num_actions
        self.main_q_network = Net(n_in, n_mid, n_out)
        self.target_q_network = Net(n_in, n_mid, n_out)
        print(self.main_q_network)
        
        self.optimizer = optim.Adam(
            self.main_q_network.parameters(), lr=0.0001)
        
    def replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        
        self.batch, self.state_batch, self.action_batch, self.reward_batch, self.non_final_next_states = self.make_minibatch()
        self.expected_state_action_values = self.get_expected_state_action_values()
        self.update_main_q_network()
        
    def decide_action(self, state, episode):
        epsilon = 0.5 * (1 / (episode + 1))
        
        if epsilon <= np.random.uniform(0, 1):
            self.main_q_network.eval()
            with torch.no_grad():
                action = self.main_q_network(state).max(1)[1].view(1, 1)
        else:
            action = torch.LongTensor([[random.randrange(self.num_actions)]])
            
        return action

    def make_minibatch(self):
        transitions = self.memory.sample(BATCH_SIZE)
        
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
        return batch, state_batch, action_batch, reward_batch, non_final_next_states
    
    def get_expected_state_action_values(self):
        self.main_q_network.eval()
        self.target_q_network.eval()
        
        self.state_action_values = self.main_q_network(self.state_batch).gather(1, self.action_batch)
        #print('state_action_values:', self.state_action_values)
        
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, self.batch.next_state)))
        
        next_state_values = torch.zeros(BATCH_SIZE)
        
        a_m = torch.zeros(BATCH_SIZE).type(torch.LongTensor)
        a_m[non_final_mask] = self.main_q_network(self.non_final_next_states).detach().max(1)[1]
            
        a_m_non_final_next_states = a_m[non_final_mask].view(-1, 1)
        
        next_state_values[non_final_mask] = self.target_q_network(
            self.non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()
        
        expected_state_action_values = self.reward_batch + GAMMA * next_state_values
        return expected_state_action_values
    
    def update_main_q_network(self):
        self.main_q_network.train()
        
        loss = F.smooth_l1_loss(self.state_action_values, self.expected_state_action_values.unsqueeze(1))
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
    def update_target_q_network(self):
        self.target_q_network.load_state_dict(self.main_q_network.state_dict())



In [17]:
class Agent:
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions)
        
    def update_q_function(self):
        self.brain.replay()
        
    def get_action(self, state, episode):
        action = self.brain.decide_action(state, episode)
        return action
    
    def memorize(self, state, action, state_next, reward):
        self.brain.memory.push(state, action, state_next, reward)
        
    def update_target_q_function(self):
        self.brain.update_target_q_network()
        

In [18]:
class Environment:
    def __init__(self):
        self.env = gym.make(ENV)
        num_states = self.env.observation_space.shape[0]
        num_actions = self.env.action_space.n
        
        self.agent = Agent(num_states, num_actions)
        
    def run(self):
        episode_10_list = np.zeros(10)
        complete_episodes = 0
        episode_final = False
        
        frames = []
        
        for episode in range(NUM_EPISODES):
            observation = self.env.reset()
            
            
            state = observation
            state = torch.from_numpy(state).type(torch.FloatTensor)
            state = torch.unsqueeze(state, 0)
            
            for step in range(MAX_STEPS):
                #if episode_final is True:
                #    frames.append(self.env.render(mode='rgb_array'))
                    
                action = self.agent.get_action(state, episode)
                
                observation_next, _, done, _ = self.env.step(action.item())
                
                if done:
                    state_next = None
                    episode_10_list = np.hstack((episode_10_list[1:], step + 1))
                    
                    if step < 195:
                        reward = torch.FloatTensor([-1.0])
                        complete_episodes = 0
                    else:
                        reward = torch.FloatTensor([1.0])
                        complete_episodes = complete_episodes + 1
                else:
                    reward = torch.FloatTensor([0.0])
                    state_next = observation_next
                    state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
                    state_next = torch.unsqueeze(state_next, 0)
                    
                self.agent.memorize(state, action, state_next, reward)
                
                self.agent.update_q_function()
                
                state = state_next
                
                if done:
                    print('%d Episode: Finished after %d steps: 10 episode avg step = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()
                    ))
                    
                    if episode % 2 == 0:
                        self.agent.update_target_q_function()
                    
                    break
                
            if episode_final is True:
                #display_frames_as_gif(frames)
                break
                    
            if complete_episodes >= 10:
                print('10 episode success')
                episode_final = True
                    

In [19]:
cartpole_env = Environment()
cartpole_env.run()

Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3_adv): Linear(in_features=32, out_features=2, bias=True)
  (fc3_v): Linear(in_features=32, out_features=1, bias=True)
)
0 Episode: Finished after 12 steps: 10 episode avg step = 1.2
1 Episode: Finished after 10 steps: 10 episode avg step = 2.2


/tmp/ipykernel_4202/314684042.py:67: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /build/python-pytorch/src/pytorch-1.9.1-opt-cuda/aten/src/ATen/native/IndexingUtils.h:29.)
  a_m[non_final_mask] = self.main_q_network(self.non_final_next_states).detach().max(1)[1]
/tmp/ipykernel_4202/314684042.py:69: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /build/python-pytorch/src/pytorch-1.9.1-opt-cuda/aten/src/ATen/native/IndexingUtils.h:29.)
  a_m_non_final_next_states = a_m[non_final_mask].view(-1, 1)
/tmp/ipykernel_4202/314684042.py:71: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /build/python-pytorch/src/pytorch-1.9.1-opt-cuda/aten/src/ATen/native/IndexingUtils.h:29.)
  next_state_values[non_final_mask] = self.target_q_network(


2 Episode: Finished after 11 steps: 10 episode avg step = 3.3
3 Episode: Finished after 10 steps: 10 episode avg step = 4.3
4 Episode: Finished after 8 steps: 10 episode avg step = 5.1
5 Episode: Finished after 11 steps: 10 episode avg step = 6.2
6 Episode: Finished after 9 steps: 10 episode avg step = 7.1
7 Episode: Finished after 9 steps: 10 episode avg step = 8.0
8 Episode: Finished after 9 steps: 10 episode avg step = 8.9
9 Episode: Finished after 11 steps: 10 episode avg step = 10.0
10 Episode: Finished after 9 steps: 10 episode avg step = 9.7
11 Episode: Finished after 9 steps: 10 episode avg step = 9.6
12 Episode: Finished after 14 steps: 10 episode avg step = 9.9
13 Episode: Finished after 16 steps: 10 episode avg step = 10.5
14 Episode: Finished after 35 steps: 10 episode avg step = 13.2
15 Episode: Finished after 15 steps: 10 episode avg step = 13.6
16 Episode: Finished after 46 steps: 10 episode avg step = 17.3
17 Episode: Finished after 48 steps: 10 episode avg step = 21.2
